<a href="https://colab.research.google.com/github/Laizechave/Vem-Aferir/blob/main/PROJETO_VEM_AFERIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto - Vem Aferir**

In [ ]:
# Instalar dependências
!pip install ipywidgets -q
!pip install pandas -q

import os
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
# Removed locale import as it's causing the error
# import locale
import pandas as pd # Import pandas for displaying the history

# Apagar banco anterior para recriar a tabela corretamente
if os.path.exists("pressao.db"):
    os.remove("pressao.db")

#  Criar o banco de dados
def criar_banco():
    conexao = sqlite3.connect("pressao.db")
    cursor = conexao.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS registros (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            dia TEXT,
            maxima INTEGER,
            minima INTEGER,
            sintomas TEXT,
            status TEXT,
            data_completa TEXT -- Added data_completa column
        )
    """)
    conexao.commit()
    conexao.close()

#  Função para analisar a pressão arterial
def analisar_pressao(maxima, minima):
    if minima < 85 and maxima < 130:
        return "✅ Normal\n👍 Continue com hábitos saudáveis, como boa alimentação e atividades físicas regulares."

    elif 85 <= minima <= 89 and 130 <= maxima <= 139:
        return "🔹 Normal limítrofe\n⚠️ Mantenha atenção! Reduza o consumo de sal e pratique atividade física com frequência."

    elif 90 <= minima <= 99 and 140 <= maxima <= 159:
        return "⚠️ Hipertensão leve (Estágio 1)\n🩺 Consulte seu médico. Evite sal, gorduras e pratique exercícios regularmente."

    elif 100 <= minima <= 109 and 160 <= maxima <= 179:
        return "⚠️⚠️ Hipertensão moderada (Estágio 2)\n⚠️ Atenção! É essencial acompanhamento médico."

    elif minima >= 110 and maxima >= 180:
        return "🚨 Hipertensão grave (Estágio 3)\n🚑 Procure atendimento médico urgente! A pressão está em nível crítico."

    elif minima < 90 and maxima >= 140:
        return "❗ Hipertensão\ Consulte seu médico para avaliação adequada."

    else:
        return "❓ Classificação fora da tabela – verifique os valores inseridos ou consulte um profissional."

# Função para salvar os dados no banco
# Removed locale setup
# locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8') # Removed

# Função para salvar os dados
def salvar_dados(b):
    clear_output(wait=True)
    try:
        maxima_valor = int(maxima.value)
        minima_valor = int(minima.value)
    except ValueError:
        print("❌ Erro: Os valores de pressão devem ser números inteiros.")
        return

    status = analisar_pressao(maxima_valor, minima_valor)

    # Gerar a data automática usando a data atual
    agora = datetime.now()
    # Using a simpler date format that does not depend on locale
    data_formatada = agora.strftime("%Y-%m-%d %H:%M:%S") # Format: YYYY-MM-DD HH:MM:SS

    conexao = sqlite3.connect("pressao.db")
    cursor = conexao.cursor()
    cursor.execute("""
        INSERT INTO registros (dia, maxima, minima, sintomas, status, data_completa)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (dia.value, maxima_valor, minima_valor, sintomas.value, status, data_formatada))
    conexao.commit()
    conexao.close()

    print("✅ Dados salvos com sucesso!")
    print(f"🗓️ Data: {data_formatada}")
    print(f"📊 Status da pressão: {status}\n")
    exibir_historico(None)

def exibir_historico(b):
    conexao = sqlite3.connect("pressao.db")
    cursor = conexao.cursor()
    cursor.execute("SELECT data_completa, dia, maxima, minima, sintomas, status FROM registros ORDER BY id DESC")
    registros = cursor.fetchall()
    conexao.close()

    if not registros:
        print("📭 Nenhum dado registrado ainda.")
        return

    # Ensure pandas is imported
    import pandas as pd

    df = pd.DataFrame(registros, columns=[
        "📅 Data completa",
        "📌 Dia da semana",
        "🔼 Pressão Máxima",
        "🔽 Pressão Mínima",
        " Sintomas",
        "📊 Classificação"
    ])

    display(df)

# Widgets de entrada
dia = widgets.Dropdown(
    options=["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"],
    description="Dia:"
)
maxima = widgets.Text(description="Máxima:")
minima = widgets.Text(description="Mínima:")
sintomas = widgets.Text(description="Sintomas:")

# Botões
botao_salvar = widgets.Button(description="Salvar Dados", button_style='success')
botao_salvar.on_click(salvar_dados)

botao_historico = widgets.Button(description="Ver Histórico", button_style='info')
botao_historico.on_click(exibir_historico)

# Interface principal
def exibir_interface():
    print("🩺 Monitor de Pressão Arterial - Vem Aferir")
    display(dia, maxima, minima, sintomas, botao_salvar, botao_historico)

# Iniciar aplicação
criar_banco()
exibir_interface()

🩺 Monitor de Pressão Arterial - Vem Aferir


Dropdown(description='Dia:', options=('Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo'), v…

Text(value='', description='Máxima:')

Text(value='', description='Mínima:')

Text(value='', description='Sintomas:')

Button(button_style='success', description='Salvar Dados', style=ButtonStyle())

Button(button_style='info', description='Ver Histórico', style=ButtonStyle())